In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Extrai de boletos (PDF) e gera CSV UTF-8 com colunas:
Linha digitável ou Código de barras* | CNPJ/CPF* | Data de agendamento* | Descrição* | Tags

Dependência: pip install pdfplumber
"""

import csv
import datetime as dt
import os
import re
import sys
from typing import Optional

# ========= CONFIG =========
CONFIG = {
    "INPUT_DIR": r"C:\Users\Igor Feu\Downloads\Boletos Iugu",
    "OUTPUT_CSV": r"C:\Users\Igor Feu\Downloads\Boletos Iugu\boletos.csv",
    "DESCRICAO": None,  # ignorado (Descrição será o nome do arquivo PDF)
    "TAGS": "Pagamento, Despesa, Cliente",
    # Data de agendamento sempre o dia de execução:
    "AGENDAMENTO_SOURCE": "today",
    "AGENDAMENTO_DATE": None,  # não usado com "today"
}
# =========================

try:
    import pdfplumber
except Exception:
    print("Erro: instale a dependência com `pip install pdfplumber`.", file=sys.stderr)
    raise

# --------- REGEX ---------
RE_DATA = re.compile(r'\b(0?[1-9]|[12][0-9]|3[01])/(0?[1-9]|1[0-2])/[12][0-9]{3}\b')
RE_CPF = re.compile(r'\b\d{3}\.\d{3}\.\d{3}-\d{2}\b|\b\d{11}\b')
RE_CNPJ = re.compile(r'\b\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}\b|\b\d{14}\b')
# Linha digitável/código de barras: aceitar com ou sem pontuação/espacos (47-48 dígitos)
RE_LINHA_DIGITAVEL_PONTUADA = re.compile(
    r'\b\d{5}\.\d{5}\s+\d{5}\.\d{6}\s+\d{5}\.\d{6}\s+\d\s+\d{14}\b'
)
RE_LINHA_DIGITAVEL_SEM_PONTUACAO = re.compile(r'\b\d{47,48}\b')


def clean_spaces(s: str) -> str:
    return " ".join(s.split()).strip()


def extract_text_from_pdf(path: str) -> str:
    with pdfplumber.open(path) as pdf:
        parts = []
        for page in pdf.pages:
            t = page.extract_text() or ""
            parts.append(t)
        return "\n".join(parts)


def find_linha_digitavel(text: str) -> Optional[str]:
    # 1) Buscar próximo ao rótulo
    lbl_idx = text.lower().find("linha digitável")
    if lbl_idx == -1:
        lbl_idx = text.lower().find("linha digitavel")
    if lbl_idx != -1:
        snippet = text[lbl_idx: lbl_idx + 300]
        md = RE_LINHA_DIGITAVEL_PONTUADA.search(snippet)
        if md:
            return clean_spaces(md.group(0))
        md2 = RE_LINHA_DIGITAVEL_SEM_PONTUACAO.search(snippet)
        if md2:
            return md2.group(0)
    # 2) Texto todo (pontuada)
    md = RE_LINHA_DIGITAVEL_PONTUADA.search(text)
    if md:
        return clean_spaces(md.group(0))
    # 3) 47–48 dígitos
    md2 = RE_LINHA_DIGITAVEL_SEM_PONTUACAO.search(text.replace(" ", ""))
    if md2:
        return md2.group(0)
    return None


def choose_cnpj_fidc(text: str) -> Optional[str]:
    """
    Retorna SEMPRE um CNPJ (do FIDC/Beneficiário), nunca CPF do cliente.
    Heurística de prioridade:
      1) CNPJ próximo de 'fidc'
      2) CNPJ próximo de 'beneficiário/beneficiario'
      3) CNPJ próximo de 'cedente'
      4) CNPJ próximo de 'sacador/avalista' (ou termos avulsos)
      5) Primeiro CNPJ do documento
    """
    text_lc = text.lower()
    anchors = [
        "fidc",
        "beneficiário", "beneficiario",
        "cedente",
        "sacador/avalista", "sacador", "avalista",
        "beneficiário/avalista", "beneficiario/avalista",
    ]
    # procurar CNPJ em uma janela ao redor do âncora
    def find_near(anchor: str, window_before: int = 60, window_after: int = 260) -> Optional[str]:
        idx = text_lc.find(anchor)
        if idx == -1:
            return None
        start = max(0, idx - window_before)
        end = min(len(text), idx + window_after)
        snippet = text[start:end]
        m = RE_CNPJ.search(snippet)
        return m.group(0) if m else None

    for a in anchors:
        cnpj = find_near(a)
        if cnpj:
            return cnpj

    # fallback: primeiro CNPJ no documento
    m = RE_CNPJ.search(text)
    if m:
        return m.group(0)

    # se nada encontrado, retorna vazio (não usa CPF)
    return None


def parse_agendamento(source: str, fixed_date: Optional[str], pdf_text: str, file_path: str) -> str:
    # Forçado para "today" conforme CONFIG atual
    if source == "today":
        return dt.date.today().strftime("%d/%m/%Y")
    if source == "fixed":
        if not fixed_date:
            raise SystemExit("AGENDAMENTO_DATE é obrigatório quando AGENDAMENTO_SOURCE='fixed'.")
        try:
            dt.datetime.strptime(fixed_date, "%d/%m/%Y")
        except Exception:
            raise SystemExit("AGENDAMENTO_DATE inválido. Use DD/MM/AAAA.")
        return fixed_date
    elif source == "filedate":
        ts = os.path.getmtime(file_path)
        return dt.datetime.fromtimestamp(ts).strftime("%d/%m/%Y")
    elif source == "vencimento":
        # fallback se alguém mudar CONFIG no futuro
        for m in RE_DATA.finditer(pdf_text):
            start = max(0, m.start() - 40)
            ctx = pdf_text[start:m.end() + 10].lower()
            if "vencimento" in ctx:
                return m.group(0)
        all_dates = [m.group(0) for m in RE_DATA.finditer(pdf_text)]
        if all_dates:
            return all_dates[0]
        raise SystemExit(f"Sem data inferível em: {file_path}")
    else:
        raise SystemExit("AGENDAMENTO_SOURCE inválido.")


def process_pdf(path: str, cfg: dict):
    text = extract_text_from_pdf(path)
    if not text.strip():
        raise ValueError(f"Nenhum texto extraído de {path}")

    linha = find_linha_digitavel(text) or ""
    doc_id = clean_spaces(linha) if linha else ""

    # CNPJ do FIDC/Beneficiário (nunca CPF do cliente)
    doc_cnpj = choose_cnpj_fidc(text) or ""

    data_ag = parse_agendamento(
        cfg["AGENDAMENTO_SOURCE"],
        cfg["AGENDAMENTO_DATE"],
        text,
        path
    )

    # Descrição = nome do arquivo PDF (com extensão)
    descricao = os.path.basename(path)

    return {
        "Linha digitável ou Código de barras*": doc_id,
        "CNPJ/CPF*": doc_cnpj,  # agora sempre CNPJ (se disponível)
        "Data de agendamento*": data_ag,
        "Descrição*": descricao,
        "Tags": cfg["TAGS"],
    }


def main():
    cfg = CONFIG
    input_dir = cfg["INPUT_DIR"]
    output_csv = cfg["OUTPUT_CSV"]

    if not os.path.isdir(input_dir):
        raise SystemExit(f"INPUT_DIR inválido: {input_dir}")

    rows = []
    for fname in sorted(os.listdir(input_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        fpath = os.path.join(input_dir, fname)
        try:
            row = process_pdf(fpath, cfg)
            rows.append(row)
        except Exception as e:
            print(f"[AVISO] Falha em {fname}: {e}", file=sys.stderr)

    fieldnames = [
        "Linha digitável ou Código de barras*",
        "CNPJ/CPF*",
        "Data de agendamento*",
        "Descrição*",
        "Tags",
    ]

    # CSV em UTF-8 (sem BOM)
    os.makedirs(os.path.dirname(os.path.abspath(output_csv)), exist_ok=True)
    with open(output_csv, "w", encoding="utf-8", newline="") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in rows:
            w.writerow(r)

    print(f"OK: {len(rows)} registro(s) gravado(s) em {output_csv}")


if __name__ == "__main__":
    main()


Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss while decompressing corrupted data
Data-loss 

OK: 14 registro(s) gravado(s) em C:\Users\Igor Feu\Downloads\Boletos Iugu\boletos.csv
